# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$

In [1]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())

Using TensorFlow backend.


#### Mode:
Software/Hardware Testing or Real-Deal?

In [2]:
trial_run = True

### Simulation Method:

In [3]:
# Random DNN
f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
# f_unknown_mode = "DNN_with_Random_Weights"
Depth_Bayesian_DNN = 2
width = 2

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.25

# Rough SDE (time 1)
# f_unknown_mode = "Rough_SDE"

## Problem Dimension

In [4]:
problem_dim = 2

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Rough SDE Meta-Parameters

In [5]:
# SDE with Rough Driver
N_Euler_Steps = 10**1
Hurst_Exponent = 0.01

def alpha(t,x):
    output_drift_update = t-x
    return output_drift_update

def beta(t,x):
    output_vol_update = (t+0.001)*np.diag(np.cos(x))
    return output_vol_update

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [6]:
train_test_ratio = .2
N_train_size = 10**1

Monte-Carlo Paramters

In [7]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**1

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [8]:
# Hyper-parameters of Cover
delta = 0.01
Proportion_per_cluster = .01

# Run Main:

In [9]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
# %run Universal_Measure_Valued_Networks_Backend.ipynb
exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

Deep Feature Builder - Ready
Deep Classifier - Ready


100%|██████████| 200/200 [00:00<00:00, 500.83it/s]

Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 4 folds for each of 5 candidates, totalling 20 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  16 out of  20 | elapsed:  2.8min remaining:   41.3s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  3.6min finished


Epoch 1/200
32/32 [==============================] - 0s 3ms/step - loss: 6.2337 - accuracy: 0.0060
Epoch 2/200
32/32 [==============================] - 0s 3ms/step - loss: 6.0418 - accuracy: 0.0200
Epoch 3/200
32/32 [==============================] - 0s 3ms/step - loss: 5.8296 - accuracy: 0.0190
Epoch 4/200
32/32 [==============================] - 0s 3ms/step - loss: 5.5402 - accuracy: 0.0390
Epoch 5/200
32/32 [==============================] - 0s 3ms/step - loss: 5.1910 - accuracy: 0.1030
Epoch 6/200
32/32 [==============================] - 0s 3ms/step - loss: 4.7075 - accuracy: 0.2130
Epoch 7/200
32/32 [==============================] - 0s 3ms/step - loss: 4.1180 - accuracy: 0.2790
Epoch 8/200
32/32 [==============================] - 0s 3ms/step - loss: 3.3240 - accuracy: 0.4710
Epoch 9/200
32/32 [==============================] - 0s 3ms/step - loss: 2.5204 - accuracy: 0.6630
Epoch 10/200
32/32 [==============================] - 0s 4ms/step - loss: 1.8212 - accuracy: 0.8220
Epoch 11/

...

#--------------------#
 Get Training Error(s)
#--------------------#


  0%|          | 0/200 [00:00<?, ?it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 200/200 [01:17<00:00,  2.59it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#


---
# Run: All Benchmarks

## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [10]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


  0%|          | 1/1000 [00:00<02:51,  5.82it/s]

#------------#
 Get Error(s) 
#------------#


  0%|          | 1/200 [00:00<00:21,  9.47it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


...

#------------#
 Get Error(s) 
#------------#


  0%|          | 1/200 [00:00<00:20,  9.83it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 200/200 [00:21<00:00,  9.34it/s]


#-----------------#
 Get Error(s): END 
#-----------------#
-----------------------
Computing Error Metrics
-----------------------


# Summary of Point-Mass Regression Models

#### Training Model Facts

In [11]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                   ENET         kRidge           GBRF           ffNN
W1           106.985520     107.294094     106.987811     106.985018
Mean      147516.058541  147516.034110  147516.057724  147516.069904
Var          106.182463     106.182463     106.182463     106.182463
Skewness       0.000000       0.000000       0.000000       0.000000
Ex_Kur         6.000000       6.000000       6.000000       6.000000


,ENET,kRidge,GBRF,ffNN
W1,106.985520,107.294094,106.987811,106.985018
Mean,147516.058541,147516.034110,147516.057724,147516.069904
Var,106.182463,106.182463,106.182463,106.182463
Skewness,0.000000,0.000000,0.000000,0.000000
Ex_Kur,6.000000,6.000000,6.000000,6.000000


#### Testing Model Facts

In [12]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                   ENET         kRidge           GBRF           ffNN
W1           108.468255     107.898307     108.461199     108.472491
Mean      142948.370517  142948.372465  142948.370525  142948.382216
Var          108.388589     108.388589     108.388589     108.388589
Skewness       0.000000       0.000000       0.000000       0.000000
Ex_Kur         6.000000       6.000000       6.000000       6.000000


#### Model Complexitie(s)

In [13]:
print(Summary_Complexity_models)
Summary_Complexity_models

        N_Params      T_Time  T_Test/T_test-MC
ENET         100  138.605759          0.006539
GBRF     2780900  211.614352          0.016204
kRidge       106  121.182766          0.056722
ffNN       90801  262.980778          0.103619


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\sigma}(x))\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to \frac1{\hat{\sigma}(x)\sqrt{2\pi}}\exp\left(\frac{-(\cdot-\hat{\mu}(x))^2}{\hat{\sigma(x)}^2}\right) \in \mathcal{G}_1\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [14]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Deep Feature Builder - Ready
Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:    6.5s remaining:    6.5s
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:   19.8s remaining:    8.5s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   24.8s finished
  0%|          | 2/1000 [00:00<00:51, 19.52it/s]

Infering Parameters for Deep Gaussian Network to train on!


100%|██████████| 1000/1000 [00:23<00:00, 42.85it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 4 folds for each of 5 candidates, totalling 20 fits


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


100%|██████████| 200/200 [00:00<00:00, 1144.51it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#---------------------------------------#
 Get Testing Errors for: Gaussian Models
#---------------------------------------#


#------------------------#
 Get Testing Error(s): END
#------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
------------------------------------------------
Updated Performance Metrics Dataframe and Saved!
------------------------------------------------
--------------------------------------------
Computing and Updating Complexity Metrics...
--------------------------------------------
-----------------------------------------------
Updated Complexity Metrics Dataframe and Saved!
-----------------------------------------------


In [ ]:
print("Prediction Quality (Updated)")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

In [ ]:
print("Model Complexities Quality (Updated)")
print(Summary_Complexity_models)
Summary_Complexity_models

# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [ ]:
# %run Mixture_Density_Network.ipynb
exec(open('Mixture_Density_Network.py').read())

100%|██████████| 1000/1000 [11:16<00:00,  1.48it/s]

Deep Feature Builder - Ready
Training Mixture Density Network (MDN): Means: Start!
Fitting 4 folds for each of 5 candidates, totalling 20 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done  16 out of  20 | elapsed:  3.3min remaining:   50.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  4.3min finished


Epoch 1/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9648 - mse: 174.5041 - mae: 9.9648 - mape: 102.3544
Epoch 2/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9648 - mse: 174.5040 - mae: 9.9648 - mape: 102.3538
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9648 - mse: 174.5040 - mae: 9.9648 - mape: 102.3530
Epoch 4/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9648 - mse: 174.5039 - mae: 9.9648 - mape: 102.3513
Epoch 5/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9648 - mse: 174.5039 - mae: 9.9648 - mape: 102.3509
Epoch 6/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9648 - mse: 174.5038 - mae: 9.9648 - mape: 102.3488
Epoch 7/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9648 - mse: 174.5037 - mae: 9.9648 - mape: 102.3475
Epoch 8/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9647 - mse: 174.5037 - mae: 9.9647 -

32/32 [==============================] - 0s 6ms/step - loss: 9.9646 - mse: 174.4999 - mae: 9.9646 - mape: 102.3106
Epoch 66/100
32/32 [==============================] - 0s 4ms/step - loss: 9.9646 - mse: 174.4998 - mae: 9.9646 - mape: 102.3103
Epoch 67/100
32/32 [==============================] - 0s 6ms/step - loss: 9.9646 - mse: 174.4997 - mae: 9.9646 - mape: 102.3109
Epoch 68/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9645 - mse: 174.4997 - mae: 9.9645 - mape: 102.3116
Epoch 69/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9645 - mse: 174.4996 - mae: 9.9645 - mape: 102.3095
Epoch 70/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9645 - mse: 174.4995 - mae: 9.9645 - mape: 102.3092
Epoch 71/100
32/32 [==============================] - 0s 5ms/step - loss: 9.9645 - mse: 174.4995 - mae: 9.9645 - mape: 102.3094
Epoch 72/100
32/32 [==============================] - 0s 6ms/step - loss: 9.9645 - mse: 174.4994 - mae: 9.9645 - mape

...

32/32 [==============================] - 0s 3ms/step - loss: 6.1952 - accuracy: 0.0620
Epoch 164/250
32/32 [==============================] - 0s 3ms/step - loss: 6.1951 - accuracy: 0.0660
Epoch 165/250
32/32 [==============================] - 0s 3ms/step - loss: 6.1951 - accuracy: 0.0670
Epoch 166/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1951 - accuracy: 0.0690
Epoch 167/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1950 - accuracy: 0.0680
Epoch 168/250
32/32 [==============================] - 0s 3ms/step - loss: 6.1950 - accuracy: 0.0650
Epoch 169/250
32/32 [==============================] - 0s 3ms/step - loss: 6.1950 - accuracy: 0.0720
Epoch 170/250
32/32 [==============================] - 0s 5ms/step - loss: 6.1950 - accuracy: 0.0610
Epoch 171/250
32/32 [==============================] - 0s 4ms/step - loss: 6.1949 - accuracy: 0.0660
Epoch 172/250
32/32 [==============================] - 0s 4ms/step - loss: 6.1949 - accuracy: 0.0630
Epoc

32/32 [==============================] - 0s 2ms/step - loss: 6.1938 - accuracy: 0.0790
Epoch 245/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1937 - accuracy: 0.0740
Epoch 246/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1937 - accuracy: 0.0700
Epoch 247/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1937 - accuracy: 0.0750
Epoch 248/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1937 - accuracy: 0.0700
Epoch 249/250
32/32 [==============================] - 0s 2ms/step - loss: 6.1936 - accuracy: 0.0760
Epoch 250/250
7/7 [==============================] - 0s 923us/step


  0%|          | 0/1000 [00:00<?, ?it/s]

Training Mixture Density Network (MDN): Mixture Coefficients: END!
#--------------------#
 Get Training Error(s)
#--------------------#


 37%|███▋      | 372/1000 [02:38<04:22,  2.39it/s]

## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

In [ ]:
# %run WrapUp_Summarizer.ipynb
exec(open('WrapUp_Summarizer.py').read())

# Result(s)

#### Chosen Kernel for GPR

In [ ]:
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))

## Model Complexities

In [ ]:
print(Summary_Complexity_models)
Summary_Complexity_models

## Prediction Quality

#### Training

In [ ]:
print(PredictivePerformance_Metrics_Train)
PredictivePerformance_Metrics_Train

#### Test

In [ ]:
print(PredictivePerformance_Metrics_Test)
PredictivePerformance_Metrics_Test

---
# Fin
---

---